In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
#import keras

import statistics

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, metrics
from tensorflow.keras.models import Model


In [2]:
tf.config.run_functions_eagerly(False)

tf.keras.backend.clear_session()
sess = tf.compat.v1.Session()
sess.as_default()

In [3]:
def my_mode( input_data ):

    freq_count = dict() 

    for d in input_data: 
        if ( d in freq_count ):
            freq_count[d] += 1 
        else: 
            freq_count[d] = 1 

    max_count = 0 
    mode_value = None 

    for k,v in freq_count.items():
        if ( v > max_count ):
            mode_value = k 
            max_count = v

    return mode_value ; 

In [4]:
def generate_data_samples( input_dataset, x_columns, y_column, sample_size, y_compress_function=my_mode):

    y_values = [] 

    # First take the y series data and apply the compression function to get a single value per sample. 
    y_series = input_dataset[y_column]; 

    for i in range(0,len(input_dataset), sample_size): 

        if ( (i + 128) <= len(y_series)):
            value = y_compress_function(y_series[i:i+sample_size])
            y_values.append(value)
            

    # Now truncate the dataset, since we need the rows to be divisible by the sample size. 
    input_dataset = input_dataset.truncate(after=len(y_values*sample_size)-1)

    input_dataset = input_dataset[x_columns]
    #input_dataset.fillna(0)
    #input_dataset.replace(np.nan, 0)
    np_x_values = input_dataset.to_numpy()
    np_x_values = np_x_values.astype(np.float32)
    np_x_values = np.nan_to_num(np_x_values, 0)
    
    #Regularize the data 
    np_x_values.shape
 
    np_x_values = np_x_values.reshape(-1, sample_size, np_x_values.shape[1] )
    #np_x_values = np_x_values.reshape(np_x_values.shape[0], np_x_values.shape[1], np_x_values.shape[2], 1) 
    
    np_y_values = np.array(y_values); 
    #np_y_values = np_y_values.reshape(np_y_values.shape[0], 1)

    return np_x_values.astype(np.float32), np_y_values.astype(np.int32) 

In [5]:
def getFileList(rootFolder, searchExp = None):

    filesFound = [] 

    for root, subdirs, files in os.walk(rootFolder): 
        for fileName in files :
            if ( (searchExp is None) or re.search(searchExp, fileName) ):
                filesFound.append(os.path.join(root,fileName)) 
    return filesFound

In [6]:
import os
import re
file_names = getFileList('./Training', searchExp = "rawData.*l.csv")
print(file_names)

['./Training\\10_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\10_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\13_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\13_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\15_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\15_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\16_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\16_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\17_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\17_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\3_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\3_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\4_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\4_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\5_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\5_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\6_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\6_BOS_LeftLowerLimb\\rawData_Day4_l.csv',

In [13]:
traindata_day1 = pd.read_csv("./12_BOS_LeftLowerLimb/rawData_Day1_l.csv")
#traindata_day1.head()
traindata_day4 = pd.read_csv("./12_BOS_LeftLowerLimb/rawData_Day4_l.csv")
#traindata_day4.head()


In [14]:
traindata_day1.dtypes

Unnamed: 0               int64
timestamp              float64
leftAnkle_X             object
leftAnkle_Y             object
leftAnkle_Z             object
leftAnkle_Magnitude     object
Dysk_Score               int64
dtype: object

Merge Patient 10_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [ ]:
trainingdata = pd.concat([traindata_day1, traindata_day4], ignore_index=True)
X_train, Y_train = generate_data_samples(trainingdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
#print(X_train[0:10])
#print(Y_train[0:10])
print(len(trainingdata))
X_train.shape
Y_train.shape
#training_data = trainingdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ] #create a dataframe for 3 axel readings
#training_labels = trainingdata['Dysk_Score'] # create a dataframe for the labels
#print(len(training_labels))

In [ ]:
class_data0 = trainingdata[(trainingdata.Dysk_Score == 0)]
class_data1 = trainingdata[(trainingdata.Dysk_Score == 1)]
class_data2 = trainingdata[(trainingdata.Dysk_Score == 2)]
class_data3 = trainingdata[(trainingdata.Dysk_Score == 3)]
class_data4 = trainingdata[(trainingdata.Dysk_Score == 4)]

print(len(class_data0))
print(len(class_data1))
print(len(class_data2))
print(len(class_data3))
print(len(class_data4))
#print(len(class_data1)/len(Y_train))

In [ ]:
print(X_train[0:5])
print(Y_train[0:5])

In [16]:
valdata_day1 = pd.read_csv("./Training/3_BOS_LeftLowerLimb/rawData_Day1_l.csv")
valdata_day4 = pd.read_csv("./Training/3_BOS_LeftLowerLimb/rawData_Day4_l.csv")
#valdata_day1.head()
valdata = pd.concat([valdata_day1, valdata_day4], ignore_index=True)
X_val, Y_val = generate_data_samples(valdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
Y_val = tf.compat.v1.one_hot(Y_val, depth=5)
#print(X)
#print(Y)
print(len(valdata))
#print(X_val[0:10])

4937348


In [ ]:
#val_class_data0 = valdata[(valdata.Dysk_Score == 0)]
val_class_data1 = valdata[(valdata.Dysk_Score == 1)]
val_class_data2 = valdata[(valdata.Dysk_Score == 2)]
val_class_data3 = valdata[(valdata.Dysk_Score == 3)]
val_class_data4 = valdata[(valdata.Dysk_Score == 4)]

#print(len(val_class_data0))
print(len(val_class_data1))
print(len(val_class_data2))
print(len(val_class_data3))
print(len(val_class_data4))

In [ ]:
valdata.head()
print(X_val[0:5])
print(Y_val[0:5])

Merge Patient 12_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [18]:
testdata_11_day1 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_11_day4 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata_18_day1 = pd.read_csv("./18_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_18_day4 = pd.read_csv("./18_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata = pd.concat([testdata_11_day1, testdata_11_day4, testdata_18_day1, testdata_18_day4], ignore_index=True)
testdata.head()

,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.430815e+09,0,-0.47059,10.1961,10.2069,0
1,1,1.430815e+09,-0.15686,-0.15686,10.1961,10.1985,0
2,2,1.430815e+09,-0,-0.31372,10.1961,10.2009,0
3,3,1.430815e+09,-0.15686,-0.15686,10.1961,10.1985,0
4,4,1.430815e+09,-0.31372,-0.15686,10.1961,10.2021,0


In [19]:
X_test, Y_test = generate_data_samples(testdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
Y_test = tf.compat.v1.one_hot(Y_test, depth=5)
print(len(testdata))
print(len(X_test))
print(len(Y_test))
print(X_test[0:5])
print(Y_test[0:5])

9957286
77791
77791
[[[ 0.000000e+00 -4.705900e-01  1.019608e+01]
  [-1.568600e-01 -1.568600e-01  1.019608e+01]
  [-0.000000e+00 -3.137200e-01  1.019608e+01]
  ...
  [-1.456200e-01 -1.537800e-01  1.005991e+01]
  [ 8.500000e-03  1.329000e-02  1.034566e+01]
  [-3.010000e-03 -1.414000e-02  1.003016e+01]]

 [[ 3.550000e-03 -1.646400e-01  1.020617e+01]
  [-1.135000e-02 -1.538400e-01  1.020715e+01]
  [-1.660900e-01 -1.611600e-01  1.034945e+01]
  ...
  [-8.870000e-03 -2.313000e-02  1.006702e+01]
  [-1.639100e-01 -1.711300e-01  1.035517e+01]
  [-1.804000e-01 -1.362100e-01  1.020069e+01]]

 [[-2.908200e-01  1.095000e-02  1.035099e+01]
  [ 3.045000e-02 -4.410000e-03  1.018003e+01]
  [-2.509000e-02  6.680000e-03  1.020216e+01]
  ...
  [-1.864600e-01 -4.440000e-02  1.004031e+01]
  [-3.323400e-01 -1.548800e-01  1.042913e+01]
  [-2.843800e-01 -1.975500e-01  1.046254e+01]]

 [[-1.464700e-01 -2.597000e-01  1.016658e+01]
  [-1.518600e-01 -3.549000e-02  1.020949e+01]
  [-1.885000e-01 -2.898000e-01  1.01

In [ ]:
#test_class_data0 = testdata[(valdata.Dysk_Score == 0)]
test_class_data1 = testdata[(testdata.Dysk_Score == 1)]
test_class_data2 = testdata[(testdata.Dysk_Score == 2)]
test_class_data3 = testdata[(testdata.Dysk_Score == 3)]
test_class_data4 = testdata[(testdata.Dysk_Score == 4)]

#print(len(val_class_data0))
print(len(test_class_data1))
print(len(test_class_data2))
print(len(test_class_data3))
print(len(test_class_data4))

In [ ]:
#testdata.shape
print(X_test[0][0])

Set up hyperparameters

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, schedules, SGD

In [8]:
class CategoricalTruePositives(tf.keras.metrics.Metric):

    def __init__(self, num_classes, #batch_size,
                 name="categorical_true_positives", **kwargs):
        super(CategoricalTruePositives, self).__init__(name=name, **kwargs)

        #self.batch_size = batch_size
        self.num_classes = num_classes    

        self.cat_true_positives = self.add_weight(name="ctp", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):     

        y_true = tf.keras.backend.argmax(y_true, axis=-1)
        y_pred = tf.keras.backend.argmax(y_pred, axis=-1)
        y_true = tf.keras.backend.flatten(y_true)

        true_poss = tf.keras.backend.sum(tf.keras.backend.cast((tf.keras.backend.equal(y_true, y_pred)), dtype=tf.float32))

        self.cat_true_positives.assign_add(true_poss)

    def result(self):

        return self.cat_true_positives

In [9]:
#METRICS = [
#  tf.keras.metrics.CategoricalAccuracy(name='acc'),
#  CategoricalTruePositives(5),
#]

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
    tf.keras.metrics.AUC(name='auc'),
    #CategoricalTruePositives(5),
]

In [11]:
#Y_train = tf.compat.v1.one_hot(Y_train, depth=5)
#print(Y_train)
#labels = tf.compat.v1.one_hot(Y_train, depth=5)
#train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
#train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation="relu", input_shape=(128, 3)),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])   

In [35]:
model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=[METRICS])

The model is a classifier with multiple classes. Thus, the fitting includes class weights. In most cases, classes are not perfectly balanced, so this addition takes this into account.

In [ ]:
Y_classes = np.unique(Y_train)
print(Y_classes)

In [14]:
class_weight = {0: 1., 1: 48., 2: 137., 3: 543., 4: 7780.,}

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
model.fit(
    x=X_train, y=Y_train, epochs=5)

In [15]:
for data_file in file_names:
    training_dataframe = pd.read_csv(data_file)
    print(data_file)
    X_train, Y_train = generate_data_samples(training_dataframe, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
    Y_train = tf.compat.v1.one_hot(Y_train, depth=5)
    print(len(X_train))
    print(len(Y_train))
    model.fit(x=X_train, y=Y_train, epochs=5,class_weight=class_weight)
    
# fit - train the model
#model.fit(X_train, Y_train, epochs=5) #class_weight=class_weight) 
        #batch_size=128,
        #validation_data=(X_val, Y_val),
#)

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./Training\10_BOS_LeftLowerLimb\rawData_Day1_l.csv
19874
19874
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/5
622/622 [==============================] - 24s 33ms/step - loss: 0.5376 - tp: 18921.0000 - fp: 448.0000 - tn: 79048.0000 - fn: 953.0000 - precision: 0.9769 - recall: 0.9520 - acc: 0.9673 - auc: 0.9966
Epoch 2/5
622/622 [==============================] - 21s 34ms/step - loss: 4140137728.0000 - tp: 18778.0000 - fp: 1048.0000 - tn: 78448.0000 - fn: 1096.0000 - precision: 0.9471 - recall: 0.9449 - acc: 0.9460 - auc: 0.9980
Epoch 3/5
622/622 [==============================] - 23s 37ms/step - loss: 33370.5898 - tp: 16242.0000 - fp: 3631.0000 - tn: 75865.0000 - fn: 3632.0000 - precision: 0.8173 - recall: 0.8172 - acc: 0.8172 - auc: 0.8981
Epoch 4/5
622/622 [==============================] - 24s 39ms/step - loss: 16.7768 - tp: 17231.0000 - fp: 2643.0000 - tn: 76853.0000 - fn: 2643.0000 

630/630 [==============================] - 26s 41ms/step - loss: 11.6409 - tp: 9170.0000 - fp: 5771.0000 - tn: 74781.0000 - fn: 10968.0000 - precision: 0.6137 - recall: 0.4554 - acc: 0.5232 - auc: 0.8278
Epoch 5/5
630/630 [==============================] - 24s 38ms/step - loss: 11.1608 - tp: 8812.0000 - fp: 5096.0000 - tn: 75456.0000 - fn: 11326.0000 - precision: 0.6336 - recall: 0.4376 - acc: 0.5055 - auc: 0.8076
./Training\16_BOS_LeftLowerLimb\rawData_Day4_l.csv
19470
19470
Epoch 1/5
609/609 [==============================] - 23s 37ms/step - loss: 1.9163 - tp: 12102.0000 - fp: 3812.0000 - tn: 74068.0000 - fn: 7368.0000 - precision: 0.7605 - recall: 0.6216 - acc: 0.6650 - auc: 0.9333
Epoch 2/5
609/609 [==============================] - 23s 38ms/step - loss: 1.7467 - tp: 13075.0000 - fp: 3085.0000 - tn: 74795.0000 - fn: 6395.0000 - precision: 0.8091 - recall: 0.6715 - acc: 0.7355 - auc: 0.9556
Epoch 3/5
609/609 [==============================] - 23s 38ms/step - loss: 1.6254 - tp: 13581

607/607 [==============================] - 22s 36ms/step - loss: 1.6330 - tp: 14148.0000 - fp: 5266.0000 - tn: 72390.0000 - fn: 5266.0000 - precision: 0.7288 - recall: 0.7288 - acc: 0.7288 - auc: 0.9718
./Training\5_BOS_LeftLowerLimb\rawData_Day1_l.csv
19126
19126
Epoch 1/5
598/598 [==============================] - 21s 35ms/step - loss: 3969.3523 - tp: 10461.0000 - fp: 8665.0000 - tn: 67839.0000 - fn: 8665.0000 - precision: 0.5470 - recall: 0.5470 - acc: 0.5470 - auc: 0.9304
Epoch 2/5
598/598 [==============================] - 22s 37ms/step - loss: 8.9912 - tp: 11352.0000 - fp: 7762.0000 - tn: 68742.0000 - fn: 7774.0000 - precision: 0.5939 - recall: 0.5935 - acc: 0.5941 - auc: 0.9476
Epoch 3/5
598/598 [==============================] - 24s 40ms/step - loss: 6.8136 - tp: 11575.0000 - fp: 7548.0000 - tn: 68956.0000 - fn: 7551.0000 - precision: 0.6053 - recall: 0.6052 - acc: 0.6053 - auc: 0.9498
Epoch 4/5
598/598 [==============================] - 23s 38ms/step - loss: 3.9686 - tp: 11767

679/679 [==============================] - 27s 39ms/step - loss: 0.5495 - tp: 20496.0000 - fp: 1208.0000 - tn: 85648.0000 - fn: 1218.0000 - precision: 0.9443 - recall: 0.9439 - acc: 0.9441 - auc: 0.99764s - loss: 0.5534 - tp: 17288.0000 - fp: 1
Epoch 5/5
679/679 [==============================] - 26s 39ms/step - loss: 0.5367 - tp: 20536.0000 - fp: 1173.0000 - tn: 85683.0000 - fn: 1178.0000 - precision: 0.9460 - recall: 0.9457 - acc: 0.9459 - auc: 0.9974
./Training\8_BOS_LeftLowerLimb\rawData_Day4_l.csv
16882
16882
Epoch 1/5
528/528 [==============================] - 23s 43ms/step - loss: 44.5920 - tp: 16511.0000 - fp: 242.0000 - tn: 67286.0000 - fn: 371.0000 - precision: 0.9856 - recall: 0.9780 - acc: 0.9809 - auc: 0.9990
Epoch 2/5
528/528 [==============================] - 21s 39ms/step - loss: 0.4883 - tp: 16247.0000 - fp: 608.0000 - tn: 66920.0000 - fn: 635.0000 - precision: 0.9639 - recall: 0.9624 - acc: 0.9632 - auc: 0.9994
Epoch 3/5
528/528 [==============================] - 21s 

In [17]:
#evaluate val_dataset before testing on test data. 
model.evaluate(X_val, Y_val)
classifications = model.predict(X_val)
print(classifications[0])
print(Y_val[0])

1206/1206 [==============================] - 14s 11ms/step - loss: 2.0124 - tp: 21498.0000 - fp: 16544.0000 - tn: 137748.0000 - fn: 17075.0000 - precision: 0.5651 - recall: 0.5573 - acc: 0.5652 - auc: 0.8404
[1. 0. 0. 0. 0.]
tf.Tensor([1. 0. 0. 0. 0.], shape=(5,), dtype=float32)


In [37]:
model.evaluate(X_test, Y_test)
classifications = model.predict(X_test)
print(classifications[0])
print(Y_test[0])

ValueError: in user code:

    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1323 test_function  *
        return step_function(self, iterator)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1314 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*arg

In [ ]:
pred = model.predict(X_test) 
pred = tf.keras.backend.argmax(pred, axis = 1)[:5] 
label = tf.keras.backend.argmax(Y_test,axis = 1)[:5] 

print(pred) 
print(label)

In [38]:
result = model.predict(X_test)
pred_size = X_train.shape[0]
print(pred_size)
preds = tf.keras.backend.argmax(result, axis=1)
preds = tf.keras.backend.one_hot(preds, 5)
print(preds)

print("\nTrue positives per classes:")
for i in range(5):
    m = tf.keras.metrics.TruePositives(name='tp')    
    m.update_state(Y_test[:, i], preds[:, i])
    print("Class {} true positives: {}".format(i, m.result()))

ValueError: in user code:

    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*

In [33]:
pred = model.predict(X_test)
pred_size = X_train.shape[0]
print(pred_size)
#pred = tf.keras.backend.argmax(pred, axis = 1)
#label = tf.keras.backend.argmax(Y_test,axis = 1) 
metrics=[METRICS]
#metrics = tf.keras.metrics.SpecificityAtSensitivity(0.5)
#metrics = metrics.result().numpy()
confusion = tf.math.confusion_matrix(labels=tf.keras.backend.argmax(Y_test,axis = 1),
                                       predictions=tf.keras.backend.argmax(pred, axis = 1), num_classes=5)
print(confusion)
print(metrics)


#print("Loss {}, Accuracy {}".format(loss, acc))
# Convert the model to the TensorFlow Lite format without quantization
#converter = tf.lite.TFLiteConverter.from_keras_model(model)
#tflite_model = converter.convert()

ValueError: in user code:

    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*

In [21]:
metrics = tf.keras.metrics.SpecificityAtSensitivity(0.5)
metrics = metrics.result().numpy()
print(metrics)

0.0


In [25]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [26]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10800     
_________________________________________________________________
dense (Dense)                (None, 32)                1632      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 12,597
Trainable params: 12,597
Non-trainable params: 0
_________________________________________________________________


In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_keras_model('CNNmodel.h5')
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmpa61jicdx\assets


In [30]:
import pathlib
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# These options are for converter optimizaitons
# Try the converter without them and explore model size and accuracy
# Then use them and reconvert the model and explore model
# size an accuracy at that point.

converter.optimizations = [tf.lite.Optimize.DEFAULT]    # Uncomment this line for Model 2 and Model 3

def representative_data_gen():                          # Uncomment the following 5 lines for Model 3
     for input_value, _ in X_test.take(100):
         yield [input_value]
converter.representative_dataset = representative_data_gen
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_model = converter.convert()
#tflite_models_file = pathlib.Path("./tmp/model1.tflite")
tflite_model_file = pathlib.Path("./tmp/model2.tflite")     # Change the filename here for Model2 and Model3!
tflite_model_file.write_bytes(tflite_model)
# Without any optimizations I got
# 87840  (model1.tflite) - the model that was created with 5 epochs from the full dataset
# With the .optimizations property set I got
# 53000(model2.tflite)
# With the .optimizations Int8 property set I got
#  (model3.tflite)


INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmptdq959d2\assets


INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmptdq959d2\assets


ValueError: Failed to parse the model: pybind11::init(): factory function returned nullptr.

In [ ]:
@tf.function
def call(x, states):
    x, states = model([x,states])
    return x, states

batch_size = 1
concrete_func = call.get_concrete_function(
    x = tf.TensorSpec(shape=[batch_size,1,128], dtype=tf.float32), 
    states = tf.TensorSpec(shape=[batch_size,1,128, 2], dtype=tf.float32))
export_dir = '/saved_model'
model.save(export_dir, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model2 = converter.convert()

In [32]:
import pathlib
from tensorflow.keras.layers import  LSTM, Input

in_dat = Input(batch_shape=(1,1,128))
in_states = Input(batch_shape=(1,1,128,2))


in_state = [in_states[:,0,:,0], in_states[:,0,:,1]]
out_dat, state_h, state_c = LSTM(128, 
                                 return_sequences=True, 
                                 return_state=True)(in_dat, initial_state=in_state)

out_states = tf.stack([state_h, state_c],axis=-1)

model = Model(inputs=[in_dat, in_states], 
                           outputs=[out_dat, out_states])


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile('test_model.tflite', 'wb') as f:
      #f.write(tflite_model2)
        f.write(netmodels)

interpreter = tf.lite.Interpreter(model_path='./test_model.tflite2')

INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmpcqn_774w\assets


INFO:tensorflow:Assets written to: C:\Users\svetl\AppData\Local\Temp\tmpcqn_774w\assets


NameError: name 'tflite_model2' is not defined

In [39]:
#Run this cell each time to test your model's accuracy (make sure to change the filename). 
#Tqdm is a Python library used to display smart progress bars that show the progress of Python code execution.
from tqdm import tqdm
# Load TFLite model and allocate tensors.
tflite_model_file = './tmp/model1.tflite'                 # Change the filename here for Model 2 and 3 with optimisations
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
print(input_index)
print(output_index)

predictions = []

for i in tqdm(range(0,100)):
    data = X_test[i]
    data = data.reshape(1, data.shape[0], data.shape[1])#, data.shape[2]) 
    #print(data.shape)
    label = Y_test[i]
    interpreter.set_tensor(input_index, data)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))
    
#    Y_test_labels.append(label.numpy()[0])
#    X_test.append(X_test)

# For model 1, 
# For model 2, 
# For model 3, 
# Note: since the it/s will depend on the computer on which the model instance is running -- it would vary a bit.

score = 0
for item in range(0,100):
  prediction=np.argmax(predictions[item])
  label = Y_test[item]
  if prediction==label:
    score=score+1

print("Out of 100 predictions I got " + str(score) + " correct")

# Model 1 - 100 Correct
# Model 2 - 99 Correct
# Model 3 - 99 Correct
# Note: training starts from a random intialization the result could be off by 1 or 2 correct.

0
31


100%|██████████| 100/100 [00:00<00:00, 365.89it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()